<a href="https://colab.research.google.com/github/Abimbola-ai/Blood-Spectorscopy/blob/main/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Research Links:
* https://www.researchgate.net/publication/258716133_Miniature_Near-Infrared_NIR_Spectrometer_Engine_For_Handheld_Applications

* https://journals.sagepub.com/doi/abs/10.1255/jnirs.1084

In [4]:
!pip install -q efficientnet

In [5]:
import pandas as pd
import ssl
from sklearn.cluster import KMeans
import numpy as np
import json
import random, string
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
# import sweetviz as sv
tqdm.pandas()
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split,cross_val_score, StratifiedKFold, StratifiedShuffleSplit, GroupKFold
from sklearn.metrics import accuracy_score,confusion_matrix,recall_score,precision_recall_curve, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
# from lightgbm import LGBMClassifier
# import lightgbm as lgb
# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
import tensorflow as tf
import tensorflow.keras.layers as L

import efficientnet.tfkeras as efn
%matplotlib inline
sns.set_style("whitegrid")
seed = 42

ssl._create_default_https_context = ssl._create_unverified_context
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [6]:
# Import the dataset
train = pd.read_csv('Updated_Train.csv', index_col=0)
test = pd.read_csv('Updated_Test.csv', index_col=0)
# train_trimmed = pd.read_csv('Data/Updated_Train_trimmed.csv', index_col=0)
# test_trimmed = pd.read_csv("Data/Updated_Test_trimmed.csv", index_col=0)
spectra_hgb = pd.read_csv('Spectra_hgb.csv', index_col=0).reset_index()
spectra_chol = pd.read_csv('Spectra_chl.csv', index_col=0)
spectra_glu = pd.read_csv('Spectra_gluc.csv', index_col=0).reset_index()
spectra_skinfatblood = pd.read_csv('Spectra_skinfatblood.csv', index_col=0)


### **Exploratory Data Analysis**

In [7]:
train.head()

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,id,std,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,donation_id
0,0.520883,0.528200,0.529795,0.537382,0.540265,0.546260,0.550882,0.558865,0.571352,0.585775,0.600688,0.612949,0.617855,0.621710,0.622049,0.623373,0.620610,0.619479,0.616463,0.612414,0.609885,0.605408,0.602115,0.597710,0.593485,0.592332,0.588680,0.587396,0.585577,0.585244,0.583597,0.584460,0.584548,0.586147,0.589067,0.591360,0.595636,0.599054,0.602856,0.607817,...,1.468505,1.454910,1.434286,1.443249,1.417403,1.418581,1.403326,1.395943,1.386288,1.381532,1.372256,1.367083,1.362999,1.353013,1.345374,1.341283,1.323804,1.331071,1.332295,1.328928,1.309800,1.328064,1.329464,1.314664,1.313583,1.313329,1.312243,1.310330,1.353629,1.336609,1.319344,1.278294,0,0.0026,39.27,44.17,low,ok,high,2250
1,0.529109,0.534852,0.541075,0.544269,0.546594,0.555183,0.561367,0.567637,0.582427,0.595801,0.608619,0.619217,0.625162,0.627111,0.629326,0.628116,0.627815,0.623644,0.620762,0.617781,0.613032,0.610404,0.605528,0.602216,0.598417,0.595489,0.593692,0.591697,0.590110,0.589316,0.589131,0.587455,0.589060,0.589305,0.591252,0.594853,0.598545,0.601338,0.605673,0.610725,...,1.477741,1.457003,1.445143,1.439036,1.434363,1.422782,1.408831,1.405730,1.394905,1.386579,1.373887,1.376208,1.360633,1.358802,1.353009,1.344997,1.332883,1.324965,1.339512,1.336343,1.322957,1.311612,1.316437,1.316180,1.316063,1.320698,1.321068,1.286939,1.301804,1.314457,1.282723,1.267465,1,0.0026,39.39,44.09,low,ok,high,2250
2,0.528434,0.532036,0.540413,0.541964,0.549347,0.555902,0.561458,0.567824,0.579568,0.593048,0.607972,0.617558,0.624398,0.626222,0.627771,0.626445,0.623953,0.622616,0.618701,0.615181,0.611624,0.608587,0.605730,0.601288,0.597154,0.595654,0.592535,0.591893,0.589885,0.588528,0.587108,0.586997,0.589043,0.588888,0.591670,0.594962,0.597498,0.600767,0.604963,0.612166,...,1.468838,1.456374,1.450920,1.438182,1.434188,1.427201,1.413866,1.403336,1.404049,1.383698,1.378195,1.374761,1.373550,1.362671,1.350157,1.344038,1.341004,1.335655,1.333660,1.325491,1.335868,1.334114,1.327512,1.339627,1.322394,1.328299,1.311948,1.310330,1.311108,1.283048,1.287730,1.266464,2,0.0026,39.50,44.00,low,ok,high,2250
3,0.530528,0.531880,0.539558,0.542433,0.549097,0.556224,0.561398,0.566963,0.577525,0.591982,0.607109,0.616252,0.622437,0.625715,0.626799,0.625676,0.625084,0.622652,0.618510,0.614896,0.611060,0.608285,0.603976,0.600319,0.596856,0.594169,0.591334,0.589620,0.589549,0.587194,0.586685,0.587205,0.587368,0.588510,0.592029,0.593472,0.597904,0.601190,0.604864,0.610652,...,1.474065,1.456521,1.456408,1.445327,1.429233,1.422914,1.422161,1.405341,1.403060,1.390475,1.381461,1.370967,1.370843,1.358100,1.362090,1.352020,1.340746,1.345123,1.332574,1.345660,1.349603,1.329325,1.331542,1.347718,1.321111,1.320285,1.303363,1.306762,1.297632,1.314811,1.314408,1.342224,3,0.0026,39.60,43.89,low,ok,high,2250
4,0.527530,0.536424,0.538207,0.541504,0.550546,0.555203,0.560394,0.567352,0.579438,0.592049,0.606031,0.617109,0.624688,0.626810,0.628163,0.627580,0.625115,0.623415

In [8]:
test.head()

,Reading_ID,donation_id,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,...,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,id,std,temperature,humidity
0,ID_KWUAQLRM,ID_2982,0.517951,0.520508,0.526852,0.531611,0.536816,0.543828,0.547761,0.554379,0.565622,0.575762,0.590253,0.600692,0.605104,0.608611,0.608169,0.606985,0.606590,0.603814,0.600409,0.597802,0.593389,0.589956,0.585429,0.583659,0.578307,0.574303,0.571248,0.570985,0.568520,0.566437,0.564261,0.563907,0.563427,0.564336,0.565472,0.567827,0.570957,0.572206,...,1.416794,1.402706,1.391447,1.385020,1.373256,1.362781,1.345200,1.341810,1.333717,1.328438,1.313325,1.305895,1.293996,1.292892,1.285608,1.275789,1.267682,1.263906,1.259461,1.249615,1.244618,1.242537,1.238426,1.233983,1.235212,1.234433,1.234971,1.223950,1.235415,1.234605,1.226695,1.210430,1.214422,1.200332,1.198297,1.248182,0,0.0027,37.09,27.22
1,ID_FN8SIVJJ,ID_2982,0.517839,0.522367,0.525186,0.534661,0.541900,0.546180,0.551687,0.556753,0.566446,0.578208,0.591039,0.601620,0.605895,0.609568,0.610188,0.608635,0.607716,0.606237,0.601746,0.597047,0.594703,0.591059,0.587887,0.582815,0.578746,0.576693,0.573624,0.571268,0.569092,0.567672,0.565814,0.564862,0.564965,0.565536,0.565475,0.567822,0.571052,0.572733,...,1.417710,1.406884,1.395073,1.387243,1.377109,1.362596,1.350712,1.343471,1.334255,1.320115,1.316767,1.305150,1.289700,1.286014,1.284353,1.274027,1.274710,1.263676,1.255490,1.256831,1.246188,1.236320,1.242136,1.234657,1.233898,1.234317,1.232710,1.238981,1.237033,1.237152,1.219627,1.216220,1.191416,1.202515,1.211942,1.242939,1,0.0027,37.20,27.17
2,ID_CI4TQCHI,ID_2982,0.517702,0.522018,0.527237,0.534374,0.541155,0.547152,0.549837,0.557513,0.566793,0.580574,0.592258,0.601226,0.608085,0.611143,0.611085,0.608700,0.608647,0.605048,0.601668,0.598030,0.595072,0.591417,0.587822,0.584160,0.579395,0.576524,0.574158,0.572215,0.569473,0.566740,0.565881,0.565431,0.565380,0.565508,0.566487,0.568591,0.570932,0.573199,...,1.414933,1.405205,1.395040,1.381880,1.377176,1.360407,1.350644,1.368027,1.338249,1.322779,1.313343,1.300640,1.295481,1.286947,1.283352,1.276702,1.270998,1.262507,1.259525,1.253063,1.247183,1.245595,1.248910,1.239302,1.236558,1.237110,1.239622,1.228925,1.240135,1.238346,1.237193,1.208477,1.208721,1.187810,1.209599,1.175631,2,0.0027,37.30,27.07
3,ID_URJMKNU7,ID_2982,0.525008,0.527439,0.536871,0.539636,0.546555,0.553183,0.558826,0.563549,0.575675,0.587214,0.597155,0.606784,0.612443,0.615604,0.616249,0.615097,0.614963,0.611250,0.608559,0.603462,0.599570,0.597532,0.593608,0.588978,0.584560,0.581973,0.578872,0.576858,0.573706,0.571340,0.570279,0.570198,0.568850,0.569583,0.570136,0.571026,0.574510,0.575988,...,1.417360,1.403515,1.401420,1.382891,1.377412,1.357813,1.352627,1.344537,1.334463,1.323303,1.313057,1.304806,1.299465,1.296792,1.286711,1.273948,1.272122,1.260676,1.255276,1.257032,1.244043,1.246127,1.238651,1.236244,1.243770,1.237752,1.229684,1.228355,1.234852,1.220159,1.218532,1.185708,1.196750,1.222484,1.177406,1.198399,3,0.0027,37.38,26.98
4,ID_HBJH67IS,ID_2982,0.520532,0.522683,0.526842,0.534634,0.539676,0.547488,0.552688,0.558355,0.568959,0

Removing donation_id 6824 from the train data as the data producer said it is not good.

In [9]:
dropped_index =  train[train["donation_id"] == 6824].index 
train_df = train.drop(dropped_index)
print(f"Train data shape is:",train_df.shape)
print(f"Test data shape is:",test.shape)

Train data shape is: (29100, 178)
Test data shape is: (3660, 176)


`Reading_ID` is in test but not in train. There are three target columns in the train dataset, `hdl_cholesterol_human`, `hemoglobin(hgb)_human`, and `cholesterol_ldl_human` - Multi target data. 

The data from each samples are repeated about 60 times at different temperatures and humidity which were said to affect the spectrum data. This might cause overfitting if fed into the model. How do I segment this data? 

In [10]:
print(f"There are",train_df["donation_id"].nunique(), "donation ids in the dataset")

There are 485 donation ids in the dataset


In [11]:
train_df.describe()

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,id,std,temperature,humidity,donation_id
count,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,...,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.00000
mean,0.508006,0.510224,0.514027,0.519668,0.526055,0.533433,0.539723,0.545135,0.553147,0.565400,0.578473,0.590122,0.597018,0.600336,0.602418,0.602800,0.601294,0.598537,0.595311,0.592134,0.588864,0.585840,0.582612,0.579085,0.576050,0.572983,0.570417,0.568512,0.566814,0.565465,0.564733,0.564479,0.564907,0.565946,0.567579,0.569962,0.572840,0.576686,0.580559,0.584676,...,1.484102,1.471450,1.460211,1.449307,1.437779,1.426966,1.415785,1.404321,1.394253,1.386659,1.374273,1.365530,1.357253,1.348600,1.340983,1.333854,1.326606,1.319885,1.313265,1.307066,1.302312,1.298047,1.294624,1.292206,1.290987,1.289934,1.288474,1.285176,1.279355,1.270425,1.262234,1.256111,1.253435,1.267373,1.253791,18366.984536,0.007529,41.248018,35.342783,6647.34433
std,0.039435,0.039200,0.039190,0.039389,0.039123,0.038855,0.039024,0.039249,0.039293,0.039577,0.039753,0.039820,0.039988,0.040063,0.040089,0.040028,0.039808,0.039554,0.039371,0.039300,0.039130,0.039084,0.039006,0.038986,0.038959,0.038849,0.038826,0.038864,0.038845,0.038811,0.038794,0.038812,0.038858,0.038916,0.038938,0.039047,0.039142,0.039334,0.039458,0.039489,...,0.078651,0.077415,0.076171,0.074999,0.073795,0.072756,0.071586,0.070813,0.070054,0.069630,0.068339,0.067723,0.067370,0.066703,0.066333,0.065884,0.065368,0.064941,0.064744,0.064297,0.064151,0.063993,0.063891,0.064004,0.064380,0.064891,0.066064,0.068361,0.072172,0.078743,0.085886,0.091853,0.096156,0.102410,0.102054,11510.875532,0.004630,2.833645,8.709672,3039.12087
min,0.237409,0.251987,0.269550,0.284636,0.302883,0.318334,0.333950,0.345124,0.361352,0.374106,0.388672,0.407299,0.416705,0.420315,0.423078,0.423848,0.423839,0.422310,0.421942,0.419578,0.416689,0.414667,0.413794,0.410312,0.408713,0.405934,0.403518,0.402689,0.401398,0.400746,0.399525,0.399710,0.399237,0.400620,0.403256,0.404547,0.407792,0.412233,0.416269,0.421089,...,1.221793,1.206966,1.196696,1.187510,1.178352,1.167833,1.160042,1.14

There seems to be no outlier in the dataset. 

In [12]:
# Check the unique values of the targets
print(train_df["hdl_cholesterol_human"].unique())
print(train_df["hemoglobin(hgb)_human"].unique())
print(train_df["cholesterol_ldl_human"].unique())

['low' 'ok' 'high']
['ok' 'low' 'high']
['high' 'ok' 'low']


This looks uniform, all target variables are classified into `low`, `ok` and `high`.

In [13]:
# Check for missing value
print(str(train_df.isnull().sum().sum()) + " missing value in train")
print(str(test.isnull().sum().sum()) + " missing value in test")

0 missing value in train
0 missing value in test


In [14]:
#Labelencode the targets
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_df["hdl_cholesterol_human"] = le.fit_transform(train_df["hdl_cholesterol_human"])
train_df["hemoglobin(hgb)_human"]= le.fit_transform(train_df["hemoglobin(hgb)_human"])
train_df["cholesterol_ldl_human"]= le.fit_transform(train_df["cholesterol_ldl_human"])

### Splitting the dataset into GroupKfold

In [15]:
group_by_donation_id = train_df.groupby(['donation_id','id']) 
group_by_donation_id.first()

absorbance0  ...  cholesterol_ldl_human
donation_id id                  ...                       
1975        12120     0.511340  ...                      2
            12121     0.502340  ...                      2
            12122     0.507582  ...                      2
            12123     0.508379  ...                      2
            12124     0.507252  ...                      2
...                        ...  ...                    ...
11906       20055     0.543958  ...                      0
            20056     0.535434  ...                      0
            20057     0.528145  ...                      0
            20058     0.533171  ...                      0
            20059     0.546692  ...                      0

[29100 rows x 176 columns]

It clearly shoes that one donor can have multiple `id`. We will do the splitting below.

# Convert the target 

In [16]:
groups_by_donation_id_list = train_df['donation_id'].copy().tolist()
y_labels = train_df[["hdl_cholesterol_human","hemoglobin(hgb)_human","cholesterol_ldl_human"]].values
y_labels

array([[1, 2, 0],
       [1, 2, 0],
       [1, 2, 0],
       ...,
       [1, 2, 0],
       [1, 2, 0],
       [1, 2, 0]])

Creating 5 folds and appended the grouped dataframes into a list. 

In [17]:
n_splits = 5
gkf = GroupKFold(n_splits = 5)

result = []   
for train_idx, val_idx in gkf.split(train_df, y_labels, groups = groups_by_donation_id_list):
    train_fold = train_df.iloc[train_idx]
    val_fold = train_df.iloc[val_idx]
    result.append((train_fold, val_fold))

train_fold_1, val_fold_1 = result[0][0],result[0][1]
train_fold_2, val_fold_2 = result[1][0],result[1][1]
train_fold_3, val_fold_3 = result[2][0],result[2][1]
train_fold_4, val_fold_4 = result[3][0],result[3][1]
train_fold_5, val_fold_5 = result[4][0],result[4][1]

# just to check if it works as intended
# sample = train_fold_1.groupby("donation_id")
# sample.get_group(2250)
# sample.get_group(2250).count()


##Baseline Modeling - Logistic Regression

In [18]:
from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc
from sklearn.model_selection import GridSearchCV,StratifiedKFold

### Fold 1

In [19]:
train_fold_1

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,id,std,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,donation_id
0,0.520883,0.528200,0.529795,0.537382,0.540265,0.546260,0.550882,0.558865,0.571352,0.585775,0.600688,0.612949,0.617855,0.621710,0.622049,0.623373,0.620610,0.619479,0.616463,0.612414,0.609885,0.605408,0.602115,0.597710,0.593485,0.592332,0.588680,0.587396,0.585577,0.585244,0.583597,0.584460,0.584548,0.586147,0.589067,0.591360,0.595636,0.599054,0.602856,0.607817,...,1.468505,1.454910,1.434286,1.443249,1.417403,1.418581,1.403326,1.395943,1.386288,1.381532,1.372256,1.367083,1.362999,1.353013,1.345374,1.341283,1.323804,1.331071,1.332295,1.328928,1.309800,1.328064,1.329464,1.314664,1.313583,1.313329,1.312243,1.310330,1.353629,1.336609,1.319344,1.278294,0,0.0026,39.27,44.17,1,2,0,2250
1,0.529109,0.534852,0.541075,0.544269,0.546594,0.555183,0.561367,0.567637,0.582427,0.595801,0.608619,0.619217,0.625162,0.627111,0.629326,0.628116,0.627815,0.623644,0.620762,0.617781,0.613032,0.610404,0.605528,0.602216,0.598417,0.595489,0.593692,0.591697,0.590110,0.589316,0.589131,0.587455,0.589060,0.589305,0.591252,0.594853,0.598545,0.601338,0.605673,0.610725,...,1.477741,1.457003,1.445143,1.439036,1.434363,1.422782,1.408831,1.405730,1.394905,1.386579,1.373887,1.376208,1.360633,1.358802,1.353009,1.344997,1.332883,1.324965,1.339512,1.336343,1.322957,1.311612,1.316437,1.316180,1.316063,1.320698,1.321068,1.286939,1.301804,1.314457,1.282723,1.267465,1,0.0026,39.39,44.09,1,2,0,2250
2,0.528434,0.532036,0.540413,0.541964,0.549347,0.555902,0.561458,0.567824,0.579568,0.593048,0.607972,0.617558,0.624398,0.626222,0.627771,0.626445,0.623953,0.622616,0.618701,0.615181,0.611624,0.608587,0.605730,0.601288,0.597154,0.595654,0.592535,0.591893,0.589885,0.588528,0.587108,0.586997,0.589043,0.588888,0.591670,0.594962,0.597498,0.600767,0.604963,0.612166,...,1.468838,1.456374,1.450920,1.438182,1.434188,1.427201,1.413866,1.403336,1.404049,1.383698,1.378195,1.374761,1.373550,1.362671,1.350157,1.344038,1.341004,1.335655,1.333660,1.325491,1.335868,1.334114,1.327512,1.339627,1.322394,1.328299,1.311948,1.310330,1.311108,1.283048,1.287730,1.266464,2,0.0026,39.50,44.00,1,2,0,2250
3,0.530528,0.531880,0.539558,0.542433,0.549097,0.556224,0.561398,0.566963,0.577525,0.591982,0.607109,0.616252,0.622437,0.625715,0.626799,0.625676,0.625084,0.622652,0.618510,0.614896,0.611060,0.608285,0.603976,0.600319,0.596856,0.594169,0.591334,0.589620,0.589549,0.587194,0.586685,0.587205,0.587368,0.588510,0.592029,0.593472,0.597904,0.601190,0.604864,0.610652,...,1.474065,1.456521,1.456408,1.445327,1.429233,1.422914,1.422161,1.405341,1.403060,1.390475,1.381461,1.370967,1.370843,1.358100,1.362090,1.352020,1.340746,1.345123,1.332574,1.345660,1.349603,1.329325,1.331542,1.347718,1.321111,1.320285,1.303363,1.306762,1.297632,1.314811,1.314408,1.342224,3,0.0026,39.60,43.89,1,2,0,2250
4,0.527530,0.536424,0.538207,0.541504,0.550546,0.555203,0.560394,0.567352,0.579438,0.592049,0.606031,0.617109,0.624688,0.626810,0.628163,0.627580,0.625115,0.623415,0.619177,0.615396,0.612

In [20]:
from sklearn.preprocessing import StandardScaler
def scaler(df):
  scale = StandardScaler()
  df_scaled = scale.fit_transform(df)
  return df_scaled


In [21]:
train_fold_1.drop(["id"], axis=1, inplace=True)
val_fold_1.drop(["id"], axis=1, inplace=True)

train_1 = train_fold_1.to_numpy()
val_1 = val_fold_1.to_numpy()
features_tr = np.array(train_1[:,:173], dtype=float)
features_val = np.array(val_1[:,:173], dtype=float)
scaler(features_tr)
scaler(features_val)
labels_tr = train_1[:, 173:176]
labels_val = val_1[:, 173:176]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


array([[ 0.31899139,  0.44945583,  0.39399203, ..., -1.06080539,
        -0.72027595,  1.05392456],
       [ 0.52385703,  0.61639937,  0.67748412, ..., -1.06080539,
        -0.67716646,  1.04464705],
       [ 0.50703238,  0.54573661,  0.66082863, ..., -1.06080539,
        -0.63764942,  1.03420985],
       ...,
       [ 0.0891952 ,  0.08271442,  0.1200951 , ...,  0.5139148 ,
         1.54656507,  0.22358697],
       [ 0.113157  ,  0.11339187,  0.13259584, ...,  0.5139148 ,
         1.55374998,  0.21894821],
       [ 0.39637775,  0.30733067,  0.26270526, ...,  0.5139148 ,
         1.55734244,  0.2201079 ]])

array([[ 0.49373258,  0.67343231,  0.48177004, ..., -0.93868479,
        -0.44335735, -0.36178351],
       [ 0.48464661,  0.54519652,  0.39029912, ..., -0.93868479,
        -0.41027645, -0.37738784],
       [ 0.71048734,  0.631831  ,  0.62341256, ..., -0.93868479,
        -0.38711983, -0.39856514],
       ...,
       [-1.13019153, -1.1196593 , -1.16581687, ...,  0.50790389,
         1.41909719,  0.20220149],
       [-1.15277667, -1.19297486, -1.14016673, ...,  0.50790389,
         1.43232955,  0.20220149],
       [-1.36861557, -1.30715974, -1.28182838, ...,  0.50790389,
         1.43894573,  0.20220149]])

In [22]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(class_weight='balanced', random_state=42)
multi_target_lr = MultiOutputClassifier(logreg, n_jobs=-1)
multi_target_lr.fit(features_tr,labels_tr)
multi_target_lr.score(features_val,labels_val)

MultiOutputClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                   random_state=42),
                      n_jobs=-1)

0.04398625429553265

In [23]:
params = {'penalty':['l1', 'l2', 'elasticnet'], 'C':[10**i for i in range(-4,5)], 'solver':['liblinear','sag']}
log_model = LogisticRegression(random_state=42, class_weight='balanced')
# log_clf = hyperparameter_model(log_model, params, df_train_scaled, y_train)
# print(log_clf.best_params_)

In [29]:
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
# with warnings.catch_warnings():
#     warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [ ]:
logreg = LogisticRegression(class_weight='balanced', random_state=42)

# param = {'C':[0.001,0.003,0.005,0.01,0.03,0.05,0.1,0.3,0.5,1,2,3,3,4,5,10,20]}
grid_clf= GridSearchCV(log_model,param_grid=params,scoring='accuracy',refit=True,cv=10)
multi_target_lr = MultiOutputClassifier(grid_clf)
multi_target_lr.fit(features_tr,labels_tr)
# multi_target_lr.score(features_val,labels_val)
print('Best roc_auc: {:.4}, with best C: {}'.format(multi_target_lr.best_score_, multi_target_lr.best_params_))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter